In [79]:
from selenium import webdriver;
from selenium.webdriver.common.keys import Keys;
import time
from datetime import datetime, timedelta
import re

In [80]:
driver = webdriver.Chrome()

In [81]:
url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do"
driver.get(url)
time.sleep(15)

In [ ]:
target_date = '2004-01-01'
while True:
    move_to_target_date_page(target_date)
    get_movie_and_rank_info()
    increase_date(target_date)

In [98]:
## 함수 모음
def increase_date(start_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    increased_date = start_datetime + timedelta(days=1)
    return increased_date

def set_target_date(target_date):
    formatted_date = target_date[2:]
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchFrom')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchTo')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
def click_more_button():
    while True:
        try:
            more_button = driver.find_element_by_css_selector('div.more  > a.btn_more')
            more_button.click()
        except:
            break
        
def get_movie_info(row_info):
    rank = row_info[0].text
    # title = row_info[1].text
    sales = row_info[3].text
    audience = row_info[7].text

    print("rank = " + rank)
    print("sales = " + sales)
    print("audience = " + audience)
    
def move_movie_template(row):
    # 영화 상세 정보를 얻기 위헤 제목 클릭
    second_td = row.find_elements_by_tag_name('td')[1]
    movie_info_link = second_td.find_element_by_css_selector('span.ellip.per90 > a')
    movie_info_link.click()
    
def get_movie_metadata(movie_info_template):
    # 영화를 클릭하면 나오는 상세 정보 템플릿에서 영화 메타데이터 가져오기
    title = movie_info_template.find_element_by_css_selector('div.hd_layer > div').text.replace("영화상영관상영중", "").strip()
    movie_info = movie_info_template.find_element_by_css_selector('dl.ovf')
    synopsys = movie_info_template.find_element_by_css_selector('p.desc_info').text
    metadata = movie_info.find_elements_by_css_selector('dt, dd')
    code = metadata[1].text
    movie_details = metadata[7].text
    release_date = metadata[11].text
    production_year = metadata[13].text

    print("title = " + title)
    print("code = " + code)
    print("movie_details = " + movie_details)
    print("release_date = " + release_date)
    print("production_year = " + production_year)
    print("synopsys = " + synopsys)
    return code

def get_director_leadactor_info(movie_info_template, code):
    # 코드 아이디로 감독, 배우 정보가 있는 staff 관련 태그 얻어내기
    staff_id = f"{code}_staff"
    staff_info = movie_info_template.find_element_by_id(staff_id)

    desctiption_info = staff_info.find_element_by_css_selector('dl.desc_info')
    directorsNLeadActors = desctiption_info.find_elements_by_css_selector('div')
    return directorsNLeadActors

def get_directors_info(directors_info):
    # 감독 이름, 코드 데이터 얻기2 - onClick()
    directors_info = directors_info.find_elements_by_css_selector('dd')

    director_set = {}
    for i in range(len(directors_info)):
        director_info = directors_info[i].find_element_by_css_selector('a')
        director_name = director_info.text
        on_click_attr = director_info.get_attribute('onClick')
        match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)
        
        if match:
            director_code = match.group(1)
            director_set[director_code] = director_name
        else:
            print("코드를 찾을 수 없습니다.")

    # 감독 딕셔너리 내용 출력
    for key, value in director_set.items():
        print(f"{key}: {value}")
        
def get_lead_actors_info(lead_actors_info):
    lead_actors_template = lead_actors_info.find_elements_by_css_selector('dd')[0]
    lead_actors_template2 = lead_actors_template.find_elements_by_css_selector('table > tbody > tr > td')[0]
    lead_actors = lead_actors_template2.find_elements_by_css_selector('a')

    lead_actor_set = {}
    for i in range(len(lead_actors)):
        lead_actor_name = lead_actors[i].text
        on_click_attr = lead_actors[i].get_attribute('onClick')
        match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)

        if match:
            lead_actor_code = match.group(1)
            lead_actor_set[lead_actor_code] = lead_actor_name
        else:
            print("코드를 찾을 수 없습니다.")

    # 주연 딕셔너리 출력
    for key, value in lead_actor_set.items():
        print(f"{key}: {value}")
        
def move_to_target_date_page(target_date):
    set_target_date(target_date)
    search_button = driver.find_element_by_css_selector('div.wrap_btn > button.btn_blue')
    search_button.click()
    click_more_button()
        
def get_movie_and_rank_info():
    movie_table = driver.find_element_by_css_selector('tbody#tbody_0')
    rows = movie_table.find_elements_by_tag_name('tr')

    print("영화 수 = " + str(len(rows)))
    
    for i in range(len(rows)):
        get_movie_info(rows[i])
        move_movie_template(rows[i])
        
        movie_info_template = driver.find_element_by_css_selector('div[tabindex="-1"]')
        code = get_movie_metadata(movie_info_template)
        directorsNLeadActors = get_director_leadactor_info(movie_info_template, code)
        directors_info = directorsNLeadActors[0]
        lead_actors_info = directorsNLeadActors[1]
        get_directors_info(directors_info)
        get_lead_actors_info(lead_actors_info)
        
        # 영화 상세정보 닫기
        driver.find_element_by_css_selector('div.hd_layer > a.close').click()


In [ ]:
## 함수 모음
def increase_date(start_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    increased_date = start_datetime + timedelta(days=1)
    return increased_date

def set_target_date(target_date):
    formatted_date = target_date[2:]
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchFrom')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
    search_start_date = driver.find_element_by_css_selector('div.tf_comm > input#sSearchTo')
    for i in range(0, 8):
        search_start_date.send_keys(Keys.BACK_SPACE)
    search_start_date.send_keys(formatted_date)
    
def click_more_button():
    while True:
        try:
            more_button = driver.find_element_by_css_selector('div.more  > a.btn_more')
            more_button.click()
        except:
            break
        
def get_movie_info(row_info):
    rank = row_info[0].text
    # title = row_info[1].text
    sales = row_info[3].text
    audience = row_info[7].text

    print("rank = " + rank)
    print("sales = " + sales)
    print("audience = " + audience)
    
def move_movie_template(row):
    # 영화 상세 정보를 얻기 위헤 제목 클릭
    second_td = row.find_elements_by_tag_name('td')[1]
    movie_info_link = second_td.find_element_by_css_selector('span.ellip.per90 > a')
    movie_info_link.click()
    
def get_movie_metadata(movie_info_template):
    # 영화를 클릭하면 나오는 상세 정보 템플릿에서 영화 메타데이터 가져오기
    title = movie_info_template.find_element_by_css_selector('div.hd_layer > div').text.replace("영화상영관상영중", "").strip()
    movie_info = movie_info_template.find_element_by_css_selector('dl.ovf')
    synopsys = movie_info_template.find_element_by_css_selector('p.desc_info').text
    metadata = movie_info.find_elements_by_css_selector('dt, dd')
    code = metadata[1].text
    movie_details = metadata[7].text
    release_date = metadata[11].text
    production_year = metadata[13].text

    print("title = " + title)
    print("code = " + code)
    print("movie_details = " + movie_details)
    print("release_date = " + release_date)
    print("production_year = " + production_year)
    print("synopsys = " + synopsys)
    return code

def get_director_leadactor_info(movie_info_template, code):
    # 코드 아이디로 감독, 배우 정보가 있는 staff 관련 태그 얻어내기
    staff_id = f"{code}_staff"
    staff_info = movie_info_template.find_element_by_id(staff_id)

    desctiption_info = staff_info.find_element_by_css_selector('dl.desc_info')
    directorsNLeadActors = desctiption_info.find_elements_by_css_selector('div')
    return directorsNLeadActors

def get_directors_info(directors_info):
    # 감독 이름, 코드 데이터 얻기2 - onClick()
    directors_info = directors_info.find_elements_by_css_selector('dd')

    director_set = {}
    for i in range(len(directors_info)):
        director_info = directors_info[i].find_element_by_css_selector('a')
        director_name = director_info.text
        on_click_attr = director_info.get_attribute('onClick')
        match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)
        
        if match:
            director_code = match.group(1)
            director_set[director_code] = director_name
        else:
            print("코드를 찾을 수 없습니다.")

    # 감독 딕셔너리 내용 출력
    for key, value in director_set.items():
        print(f"{key}: {value}")
        
def get_lead_actors_info(lead_actors_info):
    lead_actors_template = lead_actors_info.find_elements_by_css_selector('dd')[0]
    lead_actors_template2 = lead_actors_template.find_elements_by_css_selector('table > tbody > tr > td')[0]
    lead_actors = lead_actors_template2.find_elements_by_css_selector('a')

    lead_actor_set = {}
    for i in range(len(lead_actors)):
        lead_actor_name = lead_actors[i].text
        on_click_attr = lead_actors[i].get_attribute('onClick')
        match = re.search(r"mstView\('people','(\d+)'\);", on_click_attr)

        if match:
            lead_actor_code = match.group(1)
            lead_actor_set[lead_actor_code] = lead_actor_name
        else:
            print("코드를 찾을 수 없습니다.")

    # 주연 딕셔너리 출력
    for key, value in lead_actor_set.items():
        print(f"{key}: {value}")
        
def move_to_target_date_page(target_date):
    set_target_date(target_date)
    search_button = driver.find_element_by_css_selector('div.wrap_btn > button.btn_blue')
    search_button.click()
    click_more_button()
        
        
        

movie_table = driver.find_element_by_css_selector('tbody#tbody_0')
rows = movie_table.find_elements_by_tag_name('tr')

print("영화 수 = " + str(len(rows)))

# 영화 정보 얻기
rank = rows[0][0].text
# title = rows[0][1].text
sales = rows[0][3].text
audience = rows[0][7].text

print("rank = " + rank)
print("sales = " + sales)
print("audience = " + audience)

# 영화 상세 정보를 얻기 위헤 제목 클릭
second_td = row.find_elements_by_tag_name('td')[1]
movie_info_link = second_td.find_element_by_css_selector('span.ellip.per90 > a')
movie_info_link.click()

movie_info_template = driver.find_element_by_css_selector('div[tabindex="-1"]')
code = get_movie_metadata(movie_info_template)
directorsNLeadActors = get_director_leadactor_info(movie_info_template, code)
directors_info = directorsNLeadActors[0]
lead_actors_info = directorsNLeadActors[1]
get_directors_info(directors_info)
get_lead_actors_info(lead_actors_info)
    
driver.find_element_by_css_selector('div.hd_layer > a.close').click()




In [59]:
# # 감독 이름, 코드 데이터 얻기
# directors = directorsNLeadActors[0].find_elements_by_css_selector('dd')

# director_set = {}
# for i in range(len(directors)):
#     directors[i].find_element_by_css_selector('a').click()
#     director_template = driver.find_elements_by_css_selector('div[tabindex="-1"]')[1]


#     director_name = director_template.find_element_by_css_selector('div.hd_layer > div').text
#     director_detail = director_template.find_element_by_css_selector('div.item_tab > div.ovf > dl.ovf')
#     director_code = director_detail.find_elements_by_css_selector('dt, dd')[1].text
#     director_set[director_code] = director_name
#     # print("director_code = " + director_code)
#     # print("director_name = " + director_set[director_code])

#     # 뒤로가기
#     director_template.find_element_by_css_selector('a.close').click()

# # 감독 딕셔너리 내용 출력
# for key, value in director_set.items():
#     print(f"{key}: {value}")

10080327: 토드 필립스 (Todd Phillips )
